### Daytum Course: Spatial Data Analytics

#### Sivia's Bayesian Coin Example

I have a coin and you need to figure out if it is a fair coin!

* a fair coin would have a 50% probability of heads (and a 50% probability of tails)

You start with your prior assessment of my coin, a prior distribution over the probability of heads $Prob(Coin)$

* it could be based on how honest you think I am

Then you perform a set of coin tosses to build a likelihood distribution, $P(Tosses | Coin)$

* the more coin tosses, the narrower this distribution

Then you update the prior distribution with the likelihood distribution to get the posterior distribution, $P(Coin | Tosses)$.


\begin{equation}
P( Coin | Tosses ) = \frac{P( Tosses | Coin ) P( Coin )}{P( Tosses )}
\end{equation}

#### Load the Required Libraries

The following code loads the required libraries.

In [1]:
%matplotlib inline
from ipywidgets import interactive                        # widgets and interactivity
from ipywidgets import widgets                            # widgets and interactivity
import matplotlib; import matplotlib.pyplot as plt        # plotting
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator) # control of axes ticks
plt.rc('axes', axisbelow=True)                  # set axes and grids in the background for all plots
import numpy as np                                        # working with arrays
from scipy.stats import triang                            # parametric distributions
from scipy.stats import binom
from scipy.stats import norm

C:\Users\pm27995\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\pm27995\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\pm27995\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


#### Make Our Interactive Plot 

For this demonstration we will: 

* declare a set of 4 widgets in a HBox (horizontal box of widgets). 


* define a function 'f' that will read the output from these widgets and make a plot

You may have some flicker and lag.  I have not tried to optimize performance for this demonstration. 

In [2]:
# 4 slider bars for the model input
a = widgets.FloatSlider(min=0.0, max = 1.0, value = 0.5, description = 'coin bias',continuous_update=False)
d = widgets.FloatSlider(min=0.01, max = 1.0, value = 0.1, step = 0.01, description = 'coin uncert.',continuous_update=False)
b = widgets.FloatSlider(min = 0, max = 1.0, value = 0.5, description = 'prop. heads',continuous_update=False)
c = widgets.IntSlider(min = 5, max = 1000, value = 100, description = 'coin tosses',continuous_update=False)

ui = widgets.HBox([a,d,b,c],)

def f(a, b, c, d):                                        # function to make the plot   
    heads = int(c * b)
    tails = c - heads
    
    x = np.linspace(0.0, 1.0, num=1000)
    
    prior = norm.pdf(x,loc = a, scale = d)
    prior = prior / np.sum(prior)
    
    plt.subplot(221)
    
    plt.plot(x, prior,color='red',alpha=0.8,linewidth=3)  # prior distribution of coin fairness
    plt.arrow(0.48,0.04,-0.4,0,linewidth=1,facecolor='grey',edgecolor='black',head_length=.03)
    plt.annotate('Increasingly Weighted Tails',[0.10,0.042],rotation=0.0)
    plt.arrow(0.52,0.04,0.4,0,linewidth=1,facecolor='grey',edgecolor='black',head_length=.03)
    plt.annotate('Increasingly Weighted Heads',[0.55,0.042],rotation=0.0)
    plt.annotate('Double Tailed Coin',[0.02,0.02],rotation=90.0)
    plt.annotate('Double Headed Coin',[0.97,0.02],rotation=90.0)
    plt.xlim(0.0,1.0)
    plt.xlabel('Coin Bias = P(Coin Heads)'); plt.ylabel('Density'); plt.title(r'$Prior$ $Distribution$ - What do you think of the coin before the tosses?')
    plt.ylim(0, 0.05)
    plt.axvline(x=0.5,color='grey',linestyle='--');
    plt.annotate('Fair Coin',[0.49,0.02],rotation=90.0)
    plt.grid()
    plt.gca().xaxis.grid(True, which='major',linewidth = 1.0); plt.gca().xaxis.grid(True, which='minor',linewidth = 0.2) # add y grids
    plt.gca().yaxis.grid(True, which='major',linewidth = 1.0); plt.gca().yaxis.grid(True, which='minor',linewidth = 0.2) # add y grids
    plt.gca().tick_params(which='major',length=7); plt.gca().tick_params(which='minor', length=4)
    plt.gca().xaxis.set_minor_locator(AutoMinorLocator()); plt.gca().yaxis.set_minor_locator(AutoMinorLocator()) # turn on minor ticks
    
    plt.subplot(222)                                      # results from the coin tosses 
    labels = ['Heads','Tails']
    label_pos = [i for i, _ in enumerate(labels)]
    plt.bar(labels,[heads,tails],color=['red','blue'],edgecolor='black',alpha=0.8)
    plt.xticks(label_pos, labels)
    plt.ylim([0,1000]); plt.ylabel('Frequency')
    #plt.pie([heads, tails],labels = ['heads','tails'],radius = 0.5*(c/1000)+0.5, autopct='%1.1f%%', colors = ['#ff9999','#66b3ff'], explode = [.02,.02], wedgeprops = {"edgecolor":"k",'linewidth': 1}  )
    plt.title('The Experimental Result from ' + str(c) + ' Coin Tosses')
    plt.gca().yaxis.grid(True, which='major',linewidth = 1.0); plt.gca().yaxis.grid(True, which='minor',linewidth = 0.2) # add y grids
    plt.gca().tick_params(which='major',length=7); plt.gca().tick_params(which='minor', length=4)
    plt.gca().yaxis.set_minor_locator(AutoMinorLocator())
    
    likelihood = binom.pmf(heads,c,x)
    likelihood = likelihood/np.sum(likelihood)
    
    plt.subplot(223)                                      # likelihood distribution given the coin tosses
    plt.plot(x, likelihood,color='red',alpha=0.8,linewidth=3)
    plt.arrow(0.48,0.04,-0.4,0,linewidth=1,facecolor='grey',edgecolor='black',head_length=.03)
    plt.annotate('Increasingly Weighted Tails',[0.10,0.042],rotation=0.0)
    plt.arrow(0.52,0.04,0.4,0,linewidth=1,facecolor='grey',edgecolor='black',head_length=.03)
    plt.annotate('Increasingly Weighted Heads',[0.55,0.042],rotation=0.0)
    plt.annotate('Double Tailed Coin',[0.02,0.02],rotation=90.0)
    plt.annotate('Double Headed Coin',[0.97,0.02],rotation=90.0)
    plt.xlim(0.0,1.0); plt.ylim(0, 0.05)
    plt.xlabel('P(Tosses | Coin Bias)'); plt.ylabel('Density'); plt.title(r'$Likelihood$ $Distribution$ - What do the coin tosses tell you about the coin?')
    plt.axvline(x=0.5,color='grey',linestyle='--');
    plt.annotate('Fair Coin',[0.49,0.02],rotation=90.0)
    plt.grid()
    plt.gca().xaxis.grid(True, which='major',linewidth = 1.0); plt.gca().xaxis.grid(True, which='minor',linewidth = 0.2) # add y grids
    plt.gca().yaxis.grid(True, which='major',linewidth = 1.0); plt.gca().yaxis.grid(True, which='minor',linewidth = 0.2) # add y grids
    plt.gca().tick_params(which='major',length=7); plt.gca().tick_params(which='minor', length=4)
    plt.gca().xaxis.set_minor_locator(AutoMinorLocator()); plt.gca().yaxis.set_minor_locator(AutoMinorLocator()) # turn on minor ticks
    
    post = prior * likelihood
    post = post / np.sum(post)    
    
    plt.subplot(224)                                      # posterior distribution
    plt.plot(x, post,color='red',alpha=0.8,linewidth=3)
    plt.arrow(0.48,0.04,-0.4,0,linewidth=1,facecolor='grey',edgecolor='black',head_length=.03)
    plt.annotate('Increasingly Weighted Tails',[0.10,0.042],rotation=0.0)
    plt.arrow(0.52,0.04,0.4,0,linewidth=1,facecolor='grey',edgecolor='black',head_length=.03)
    plt.annotate('Increasingly Weighted Heads',[0.55,0.042],rotation=0.0)
    plt.annotate('Double Tailed Coin',[0.02,0.02],rotation=90.0)
    plt.annotate('Double Headed Coin',[0.97,0.02],rotation=90.0)
    plt.xlim(0.0,1.0); plt.ylim(0, 0.05)
    plt.xlabel('P(Coin Bias | Tosses)'); plt.ylabel('Density'); plt.title(r"$Posterior$ $Distribution$ - What do you think of the coin after the tosses?")    
    plt.axvline(x=0.5,color='grey',linestyle='--');
    plt.annotate('Fair Coin',[0.49,0.02],rotation=90.0)
    plt.grid()
    plt.gca().xaxis.grid(True, which='major',linewidth = 1.0); plt.gca().xaxis.grid(True, which='minor',linewidth = 0.2) # add y grids
    plt.gca().yaxis.grid(True, which='major',linewidth = 1.0); plt.gca().yaxis.grid(True, which='minor',linewidth = 0.2) # add y grids
    plt.gca().tick_params(which='major',length=7); plt.gca().tick_params(which='minor', length=4)
    plt.gca().xaxis.set_minor_locator(AutoMinorLocator()); plt.gca().yaxis.set_minor_locator(AutoMinorLocator()) # turn on minor ticks

    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.6, wspace=0.2, hspace=0.3)
    plt.show()

interactive_plot = widgets.interactive_output(f, {'a': a, 'd': d, 'b': b, 'c': c})
interactive_plot.clear_output(wait = True)                # reduce flickering by delaying plot updating


### Bayesian Coin Example from Sivia, 1996, Data Analysis: A Bayesian Tutorial

#### Michael Pyrcz, Associate Professor, The University of Texas at Austin 

##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1) | [GeostatsPy](https://github.com/GeostatsGuy/GeostatsPy)

### The Problem

What is the probability that Dr. Pyrcz has a fair coin? Let's solve this with Bayesian updating. The inputs:

* **coin bias**: expectation of your prior distribution (probability of heads)
,  **coin uncert.**: standard deviation of your prior distribution 

* **prop. heads**: proportion of heads in the coin toss experiment, **coin tosses**: number of coin tosses in the coin toss experiment

In [3]:
display(ui, interactive_plot)                            # display the interactive plot

Output()

#### Comments

This was a simple demonstration of interactive plots in Jupyter Notebook Python with the ipywidgets and matplotlib packages. 

I have many other demonstrations on data analytics and machine learning, e.g. on the basics of working with DataFrames, ndarrays, univariate statistics, plotting data, declustering, data transformations, trend modeling and many other workflows available at https://github.com/GeostatsGuy/PythonNumericalDemos and https://github.com/GeostatsGuy/GeostatsPy. 
  
I hope this was helpful,

*Michael*

#### The Author:

### Michael Pyrcz, Associate Professor, University of Texas at Austin 
*Novel Data Analytics, Geostatistics and Machine Learning Subsurface Solutions*

With over 17 years of experience in subsurface consulting, research and development, Michael has returned to academia driven by his passion for teaching and enthusiasm for enhancing engineers' and geoscientists' impact in subsurface resource development. 

For more about Michael check out these links:

#### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

<i>&copy; Copyright daytum 2021. All Rights Reserved</i>